$O(logN)$의 삽입, 삭제 수행시간을 갖는 AVL 트리와 레드블랙 트리보다 더 빠른 연산을 위해,  
키와 1차원 리스트의 인덱스 관계를 이용하여 키(항목)를 저장한다.  
키를 1차원 리스트의 인덱스로 그대로 사용하면 메모리 낭비가 심해질 수 있으므로,  
간단한 함수를 사용해 변환한 값을 리스트의 인덱스로 이용하여 항목을 저장하는 것을 해싱(Hashing)이라고 한다.  
해싱에 사용되는 함수를 **해시함수(Hash Function)**, 해시함수가 계산한 값을 **해시값(Hash value) 또는 해시주소**라고 하며,  
항목이 해시값에 따라 저장되는 1차원 리스트를 **해시테이블(Hash Table)**이라 한다.  

아무리 우수한 해시함수를 사용하더라도 서로 다른 키들이 동일한 해시값을 갖는 **충돌(Collision)**을 막을 순 없다.  
이에 다양한 충돌 해결 방법이 있는데, 크게 **개방주소방식(Open Addressing)**과 **폐쇄주소방식(Closed Addressing)**으로 나뉜다.  
개방주소방식은 충돌된 키들을 해시테이블 전체를 열린 공간으로 여겨 비어 있는 곳을 찾아 항목을 저장하는 방식이다.  
즉 충돌이 발생한 키를 원래의 해시값과 다른 곳에 저장한다는 의미이다.  
반면 폐쇄주소방식은 해시값에 대응되는 해시테이블 워소에 반드시 키를 저장한다. 따라서 충돌이 발생한 키들을 동일한 해시주소에 저장한다.  


#### 해시함수
이상적인 해시함수는 키들을 균등하게 해시테이블의 인덱스로 변환하는 함수이다.  
키의 가장 앞 부분 몇자리 등 단순한 방식의 해시함수는 많은 충돌을 야기할 수 있다.  
단순하면서 동시에 키들을 균등하게 변환하는 함수가 해시함수로서 바람직하다.  
1. 중간제곱(Mid-Square)함수
    - 키를 제곱한 후, 적절한 크기의 중간부분을 해시값으로 사용한다.
2. 접기(Folding)함수
    - 큰 자릿수를 갖는 십진수를 키로 사용하는 경우, 몇 자리씩 일정하게 끊어서 만든 숫자들의 합을 이용해 해시값을 만든다.  
    - 123456789012라면 1234 + 5678 + 9012 = 15924를 계산한 후 해시테이블 크기가 1,000이라면 15924에서 3자리 수만을 해시값으로 사용.  
3. 곱셈(Multiplicative)함수
    - 1보다 작은 실수 delta를 키에 곱하여 얻은 숫자의 소수부분을 테이블 크기 M과 곱한다. 
    - 나온 값의 정수 부분을 해시값으로 사용한다.
    - $h(key) = ((key * delta) \% 1) * M$
4. 나눗셈(Division)함수
    - 키를 소수(Prime) M으로 나눈 뒤 나머지를 해시값으로 사용.
    - $h = key \% M$
    - 제수로 소수를 사용하면 나눗셈 연산 시 키들을 균등하게 인덱스로 변환시키는 성질을 갖음..

## 개방주소방식(Open Addressing)
#### 1. 선형조사(Linear Probing)
- 충돌이 일어난 원소에서부터 순차적으로 검색하여 처음 발견한 empty 원소에 충돌이 일어난 키를 저장.
- h = i라면 해시테이블 a[i], a[i + 1], a[i + 2], ...를 차례로 검색하여 최초로 찾아낸 empty 원소에 key를 저장.
- $(h(key) + j)\% M, j = 0, 1, 2, 3, ...$
- 순차탐색으로 empty 원소를 찾는 중 해시테이블의 키들이 빈틈없이 뭉쳐지는 **1차 군집화(Primary Clustering)**이 발생.
- 탐색, 삽입, 삭제 연산 수행 시 군집된 키들을 순차적으로 방문해야 하는 문제점..

In [1]:
# Linear Probing
class LinearProbing:
    def __init__(self, size):
        self.M = size
        self.a = [None] * size
        self.d = [None] * size
        
    def hash(self, key):
        return key % self.M
    
    def put(self, key, data):  # 삽입 연산
        initial_position = self.hash(key)
        i = initial_position
        j = 0
        while True:
            if self.a[i] == None:  # 비어있는 곳.
                self.a[i] = key
                self.d[i] = data
                return
            if self.a[i] == key:   # 이미 키가 있으면 data만 갱신
                self.d[i] = data
                return
            j += 1
            i = (initial_position + j) % self.M # 다음 원소 검사를 위해..
            if i == initial_position:  # 다음 위치가 초기 위치와 같으면 저장 실패..
                break
                
    def get(self, key):
        initial_position = self.hash(key)
        i = initial_position
        j = 1
        while self.a[i] != None:
            if self.a[i] == key:
                return self.d[i]
            i = (initial_position + j) % self.M
            j += 1
            if i == initial_position:
                return None
        return None
    
    def print_table(self):
        for i in range(self.M):
            print(f'{str(i):4}', ' ', end='')
        print()
        for i in range(self.M):
            print(f'{str(self.a[i]):4}', ' ', end='')
        print()

In [2]:
t = LinearProbing(13)
t.put(25, 'grape')
t.put(37, 'apple')
t.put(18, 'banana')
t.put(55, 'cherry')
t.put(22, 'mango')
t.put(35, 'lime')
t.put(50, 'orange')
t.put(63, 'watermelon')
print('탐색 결과:')
print('50의 data = ', t.get(50))
print('63의 data = ', t.get(63))
print('해시테이블:')
t.print_table()

탐색 결과:
50의 data =  orange
63의 data =  watermelon
해시테이블:
0     1     2     3     4     5     6     7     8     9     10    11    12    
50    63    None  55    None  18    None  None  None  22    35    37    25    


#### 2. 이차조사(Quadratic Probing)
- 선형조사와 근본적으로 동일..
- 충돌 후 1차원 리스트 a에서 $(h(key) + j^2) \%M, j = 0, 1, 2, 3, ...$으로 선형조사보다 더 멀리 떨어진 곳에서 emtpy원소를 찾음
- 1차 군집화 문제는 해결하지만, 같은 해시값을 갖는 서로 다른 키들인 동의어(Synonym)들이 똑같은 점프 시퀀스를 따라 저장하므로 결국 또 다른 형태의 군집화인 2차 군집화를 야기한다.

In [4]:
class QuadProbing:
    def __init__(self, size):
        self.M = size
        self.a = [None] * size
        self.d = [None] * size
        self.N = 0 # 항목 수
        
    def hash(self, key):
        return key % self.M
    
    def put(self, key, data):
        initial_position = self.hash(key)
        i = initial_position
        j = 0
        while True:
            if self.a[i] == None:
                self.a[i] = key
                self.d[i] = data
                self.N += 1
                return
            if self.a[i] == key:
                self.d[i] = data
                return
            j += 1
            i = (initial_position + j * j) % self.M
            if self.N > self.M:
                break
                
    def get(self, key):
        initial_position = self.hash(key)
        i = initial_position
        j = 1
        while self.a[i] != None:
            if self.a[i] == key:
                return self.d[i]
            i = (initial_position + j * j) % self.M
            j += 1
        return None
                

#### 3. 랜덤조사(Random Probing)
- 점프 시퀀스를 무작위화하여 empty 원소를 찾는 충돌 해결 방법이다.  
- 파이썬의 dictionary는 랜덤조사 기반.
- 의사 난수 생성기(Pseudo Random Number Generator) 사용.
- 2차 군집화와 유사한 형태의 3차 군집화 발생..(Tertiary Clustering)

In [5]:
import random
class RandProbing:
    def __init__(self, size):
        self.M = size
        self.a = [None] * size
        self.d = [None] * size
        self.N = 0 # 항목 수
        
    def hash(self, key):
        return key % self.M
    
    def put(self, key, data):
        initial_position = self.hash(key)
        i = initial_position
        random.seed(1000)
        while True:
            if self.a[i] == None:
                self.a[i] = key
                self.d[i] = data
                self.N += 1
                return
            if self.a[i] == key:
                self.d[i] = data
                return
            j = random.randint(1, 99)
            i = (initial_position + j) % self.M
            if self.N > self.M:
                break
                
    def get(self, key):
        initial_position = self.hash(key)
        i = initial_position
        random.seed(1000)
        while self.a[i] != None:
            if self.a[i] == key:
                return self.d[i]
            i = (initial_position + random.randint(1, 99)) % self.M
        return None        

#### 4. 이중해싱(Double Hashing)
- 2개의 해시함수를 사용하여 충동을 해결. 충돌이 나면 다른 해시함수의 해시값을 이용하여 원소 검사
- 두 해시함수 중 하나는 기본적인 해시함수 h(key)로 키를 해시테이블의 인덱스로 변환.
- 다른 함수 d(key)는 충돌 발생 시 다음 위치를 위한 점프 크기를 다음의 규칙에 따라 정함
- $h(key) + j * d(key)) mod M, j = 0, 1, 2, ...$
- 이중해싱은 동의어들이 저마다 제 2의 해시함수를 갖기 때문에 점프 시퀀스가 일정하지 않다.
- 모든 군집화 문제를 해결하는 충돌 해결 방법!
- 제 2의 함수 d(key)는 점프 크기를 정하는 함수이므로 0을 리턴해선 안된다. 
- d(key)의 값이 해시테이블의 크기 M과 서로소 관계일 때 좋은 성능..
- M을 소수로 선택하면 자연히 만족됨..

In [12]:
class DoubleHashing:
    
    def __init__(self, size):
        self.M = size
        self.a = [None] * size
        self.d = [None] * size
        self.N = 0
        
    def hash1(self, key):
        return key % self.M
    
    def hash2(self, key):
        return 9 - (key % 9)
    
    def put(self, key, data):
        initial_position = self.hash1(key)
        i = initial_position
        while True:
            if self.a[i] == None:
                self.a[i] = key
                self.d[i] = data
                self.N += 1
                return
            if self.a[i] == key:
                self.d[i] = data
                return
            i = (initial_position + self.hash2(key)) % self.M
            if self.N > self.M:
                break
                
    def get(self, key):
        initial_position = self.hash1(key)
        i = initial_position
        while self.a[i] != None:
            if self.a[i] == key:
                return self.d[i]
            i = (initial_position + self.hash2(key)) % self.M
        return None
    
    def print_table(self):
        for i in range(self.M):
            print(f'{str(i):4}', ' ', end='')
        print()
        for i in range(self.M):
            print(f'{str(self.a[i]):4}', ' ', end='')
        print()

In [13]:
t = DoubleHashing(13)
t.put(25, 'grape')
t.put(37, 'apple')
t.put(18, 'banana')
t.put(55, 'cherry')
t.put(22, 'mango')
t.put(35, 'lime')
t.put(50, 'orange')
t.put(63, 'watermelon')
print('탐색 결과:')
print('50의 data = ', t.get(50))
print('63의 data = ', t.get(63))
print('해시테이블:')
t.print_table()

탐색 결과:
50의 data =  orange
63의 data =  watermelon
해시테이블:
0     1     2     3     4     5     6     7     8     9     10    11    12    
None  None  50    55    None  18    None  63    None  22    35    37    25    


## 폐쇄주소방식(Closed Addressing)
### 체이닝(Chaining)
- 키에 대한 해시값에 대응되는 곳에만 키를 저장.  
- 충돌이 발생한 키들은 한 위치에 모아 저장됨. 
- 체이닝은 해시테이블 크기인 M개의 단순연결리스트를 가지며, 키를 해시값에 대응되는 연결리스트에 저장하는 해시방식이다.    
- 체이닝은 연결리스트로 구현되어 레퍼런스가 차지하는 공간이 추가로 필요
- 개방주소방식처럼 해시테이블의 empty원소를 찾는 오버헤드가 없고, 어떠한 군집화 현상도 없다.
- 구현이 간결하여 실제로 가장 많이 활용된다.
- 테이블 크기인 M이 항목의 수 N보다 너무 크면 많은 연결리스트들이 empty가 되고, M이 너무 작으면 연결리스트들의 길이가 너무 길어져 해시 성능이 매우 낮아진다.  
- **일반적으로 M을 소수로 선택, M = N / 10정도이면, 즉 연결리스트의 평균 길이가 약 10 정도일 때 좋은 성능을 보임.**

In [16]:
class Chaining:
    class Node:
        def __init__(self, key, data, link):
            self.key = key
            self.data = data
            self.next = link
    
    def __init__(self, size):
        self.M = size
        self.a = [None] * size
        
    def hash(self, key):
        return key % self.M
    
    def put(self, key, data):
        i = self.hash(key)
        p = self.a[i]
        while p != None:
            if key == p.key:
                p.data = data
                return
            p = p.next
        self.a[i] = self.Node(key, data, self.a[i])  # 연결 리스트 맨 앞에 삽입!
        
    def get(self, key):
        i = self.hash(key)
        p = self.a[i]
        while p != None:
            if key == p.key:
                return p.data
            p = p.next
        return None
    
    def print_table(self):
        for i in range(self.M):
            print('%2d' % (i), end='')
            p = self.a[i]
            while p != None:
                print('-->[', p.key, ',', p.data, ']', end='')
                p = p.next
            print()

In [17]:
t = Chaining(13)
t.put(25, 'grape')
t.put(37, 'apple')
t.put(18, 'banana')
t.put(55, 'cherry')
t.put(22, 'mango')
t.put(35, 'lime')
t.put(50, 'orange')
t.put(63, 'watermelon')
print('탐색 결과:')
print('50의 data = ', t.get(50))
print('63의 data = ', t.get(63))
print('해시테이블:')
t.print_table()

탐색 결과:
50의 data =  orange
63의 data =  watermelon
해시테이블:
 0
 1
 2
 3-->[ 55 , cherry ]
 4
 5-->[ 18 , banana ]
 6
 7
 8
 9-->[ 35 , lime ]-->[ 22 , mango ]
10
11-->[ 63 , watermelon ]-->[ 50 , orange ]-->[ 37 , apple ]
12-->[ 25 , grape ]


### 기타 해싱
#### 1. 2방향 체이닝(Two-way Chaining)
- 2개의 해시함수를 이용하여 연결리스트의 길이가 짧은 쪽에 새 키(항목)를 저장.  
- Node를 가리키는 레퍼런스 외에도 연결리스트의 길이(length)를 가지고 있다.
- 2방향 체이닝은 2개의 해시함수를 계산해야 하고 연결리스트의 길이를 비교해야하며, 탐색을 위해선 두 연결리스트를 탐색해야 하는 경우도 발생
- 총 N개의 키를 2-방향 체이닝으로 저장하였을 때, 연결리스트의 평균 길이는 $O(loglogN)$으로 매우 짧아서 실제로 매우 좋은 성능..
#### 2. 뻐꾸기 해싱(Cuckoo Hashing)
- 2개의 해시함수와 각 함수에 대응되는 해시테이블을 이용해 충돌이 발생하면 그 곳에 있는 키를 쫓아냄.
- 2개의 해시함수와 2개의 해시테이블에 키들을 아래 알고리즘에 따라 저장. 해시함수 h(key)는 h-table, d(key)는 d-table을 위한 것.
    1. key = new_key
    2. h(key) = i를 계산하여 h-table[i]에 key 저장
    3. if key가 저장된 원소가 비어 있으면: 삽입종료
    4. else: # key가 저장되면서 그 자리에 있떤 키를 쫓아낸 경우, 쫓겨난 키를 old_key라 하자
    5. if old_key가 있던 테이블이 h-table이면:
        - d(old_key) = j를 계산하여, d-table[j]에 old_key를 저장.
    6. else: # old_key가 d-table에 있었다면
        - h(old_key) = j를 계산하여, h-table[j]에 old_key 저장.
    7. key = old_key, go to step 3
- 삽입 도중 싸이클(Cycle)이 발생하면 삽입 과정이 종료되지 않음. 재해시를 수행해야함.
- 탐색과 삭제를 O(1)에 보장..최대 2번의 해시함수 계산으로 각각의 테이블 원소를 찾아 각 연산을 처리..
- 삽입은 높은 확률로 O(1) 시간에 수행 가능.

#### 재해시(Rehash)
어떤 해싱 방법도 해시테이블에 비어있는 원소가 적으면, 삽입에 실패하거나 해시 성능이 급격히 저하되는 현상을 피할 수 없다.  
이 경우 해시테이블을 확장시키고 새로운 해시함수를 사용하여 모든 키들을 새로운 해시테이블에 다시 저장하는 재해시(Rehash)가 필요.  
재해시는 오프라인(Off-line)에서 이루어지고 모든 키들을 다시 저장해야 하므로 O(N)시간 소요.  
재해시 수행 여부는 적재율(Load Factor)에 따라 결정..   
적재율 alpha = (테이블에 저장된 키(항목)의 수 N) / (테이블 크기 M)  
일반적으로 alpha >= 0.75가 되면 해시테이블 크기를 2배로 늘리고, alpha <= 0.25가 되면 1/2로 줄임.  

#### 동적 해싱(Dynamic Hashing)
- 대용량의 데이터베이스를 위한 해시방법.  
- 재해시를 수행하지 않고 동적으로 해시테이블의 크기를 조절..
- 확장 해싱(Extendible Hashing), 선형 해싱(Linear Hashing)이 있다.
1. 확장 해싱
    - 디렉터리를 주기억장치에 저장하고, 데이터는 디스크 블록 크기의 버킷 단위로 저장
    - 버킷은 키를 저장하는 곳. 버킷에 overflow가 발생하면 새 버킷을 만들어 나누어 저장. 이 버킷들을 가리키던 디렉터리는 2배 확장.
2. 선형 해싱
    - 디렉터리 없이 삽입할 때 버킷을 순서대로 추가하는 방식
    - 추가되는 버킷은 삽입되는 키가 저장되는 버킷과 무관하게 순차적으로 추가된다.
    - 만일 삽입되는 버킷에 저장공간이 없으면 overflow 체인에 새 키를 삽입.
    - 체인은 단순연결리스트로서 overflow된 키들을 임시로 저장
    - 나중에 버킷이 추가되면 overflow 체인의 키들을 버킷으로 이동
    - 디렉터리를 사용하지 않는다는 장점, 인터렉팁 응용에 적합.